In [1]:
import undetected_chromedriver as uc 
import os
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By

In [2]:
""" Test avec Seloger"""
pages = []


## Set Chrome Options
options = uc.ChromeOptions()
#options.headless=True
#options.add_argument('--headless')
#options.add_argument(f'--proxy-server={PROXY}')
#options.add_argument(f"user-agent={my_user_agent}")

## Create Undetected Chromedriver with Proxy
driver = uc.Chrome(options=options)


url = "https://www.seloger.com/list.htm?projects=1&types=2%2C1&places=%5B%7B%22inseeCodes%22%3A%5B750056%5D%7D%5D&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&LISTING-LISTpg=1"
## Send Request
driver.get(url)
annonces = driver.find_elements(By.XPATH, '//a[@data-testid="sl.explore.coveringLink"]')
liste_liens = []
# Récupérez les liens des annonces
for lien in annonces:
    lien_annonce = lien.get_attribute('href')
    liste_liens.append(lien_annonce)
    print(lien_annonce)

driver.quit()



https://www.seloger.com/annonces/locations/appartement/paris-1er-75/seine-et-berges/162852333.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[750056]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S
https://www.seloger.com/annonces/locations/appartement/paris-8eme-75/parc-monceau/166978385.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[750056]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S
https://www.seloger.com/annonces/locations/appartement/paris-18eme-75/montmartre/167290247.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[750056]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S
https://www.seloger.com/annonces/locations/appartement/paris-16eme-75/auteuil-sud/166959135.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[750056]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S
https://www.seloger.com

In [3]:
liste_liens[0]

'https://www.seloger.com/annonces/locations/appartement/paris-1er-75/seine-et-berges/162852333.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[750056]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S'

In [49]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

url_annonce = liste_liens[2]
df = pd.DataFrame()
options = uc.ChromeOptions()
driver = uc.Chrome(options=options)
driver.get(url_annonce)





try:
    # Utilisez une attente explicite pour attendre que la page accepter les cookies s'affiche
    continuer_sans_accepter_button = WebDriverWait(driver, 100).until(
        EC.presence_of_all_elements_located((By.XPATH, "//span[@class='didomi-continue-without-agreeing']"))
    )

    #Continue sans accepter les cookies
    continuer_sans_accepter_button[0].click()

    # Wait a bit for the text to fully load after clicking
    time.sleep(2)
        
except Exception as e:
    print(f"Could not click 'Continuer sans accepter button: {e}")


try: 
    voir_tout_button = driver.find_element(By.XPATH, "//button[@data-test='show-detail-feature-button-desktop']")
    voir_tout_button.click()
    time.sleep(1)

except Exception as e:
    print(f"Could not click 'Voir toutes les caractéristiques' button: {e}")

#Titre de l'annonce:
titre_annonce = driver.find_elements(By.XPATH, '//div[@class="Summarystyled__Title-sc-1u9xobv-4 dbveQQ"]')[0].text
titre_annonce_dict = {"Titre de l'annonce" : titre_annonce}

#Prix:
prix = driver.find_elements(By.XPATH, '//span[@class="global-styles__TextNoWrap-sc-1gbe8ip-6 gxurQr"]')[1].text #à voir si c'est toujours la 2e span 
df["Prix"] = prix

#Autres infos:
#Ordre: Nb pièces, (Nb chambres), Surface, Nb etages, Balcon, Terasse, Jardin, Garage, Ascenceur, Piscine  ?Parking?
autres_infos_WebElement = driver.find_elements(By.XPATH, '//div[@class="Tags__TagContainer-sc-edpl7u-0 EPxew"]')
autres_infos = [element.text for element in autres_infos_WebElement]
df["Autres Infos"] = ", ".join(autres_infos)

#Quartier:
quartier = driver.find_elements(By.XPATH, '//span[@data-test="neighbourhood"]')[0].text
df["Quartier"] = quartier

#Ville + code postal:
ville_cp = driver.find_elements(By.XPATH, '//span[@class="Localizationstyled__City-sc-gdkcr2-1 bgtLnh"]')[0].text
df["Ville et CP"] = ville_cp


#Description:
description = driver.find_elements(By.XPATH, "//div[@class='ShowMoreText__UITextContainer-sc-1swit84-0 fDeZMv']")[0].text
df["Description"] = description

#Caractéristiques:
#Récupère les blocs de caractéristiques (ex: Extérieur, Cadre et situation...)
blocs_caract = driver.find_elements(By.CSS_SELECTOR, "div.TitledDescription__TitledDescriptionContainer-sc-p0zomi-0")


for bloc in blocs_caract:
    titre_caract_l = bloc.find_elements(By.CSS_SELECTOR, "div.feature-title") 
    if not titre_caract_l: #Vérifie si titre_caract est vide
        continue
    titre_caract = titre_caract_l[0].text #Récupère le titre des caractéristiques
    caract_list = bloc.find_elements(By.CSS_SELECTOR, "div.GeneralFeaturesstyled__TextWrapper-sc-1ia09m5-3")
    df[titre_caract] = ", ".join([caract.text for caract in caract_list])



driver.quit()







Could not click 'Voir toutes les caractéristiques' button: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@data-test='show-detail-feature-button-desktop']"}
  (Session info: chrome=120.0.6099.234); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   undetected_chromedriver             0x0000000100b5acc8 undetected_chromedriver + 4844744
1   undetected_chromedriver             0x0000000100b52243 undetected_chromedriver + 4809283
2   undetected_chromedriver             0x000000010072077d undetected_chromedriver + 411517
3   undetected_chromedriver             0x0000000100770ee0 undetected_chromedriver + 741088
4   undetected_chromedriver             0x0000000100771091 undetected_chromedriver + 741521
5   undetected_chromedriver             0x00000001007bcab4 undetected_chromedriver + 1051316
6   undetected_chromedriver             0x0000

In [50]:
df

,Titre de l'annonce
